#### Імпорт необхідних бібліотек

In [1]:
import numpy as np
import pandas as pd

from typing import Dict
from ortools.linear_solver import pywraplp

#### Створення DataFrame на основі CSV файлу

In [2]:
df = pd.read_csv("data.csv")
df.set_index("ISP", inplace = True)
df

,d_speed,u_speed,latency,cost,reliability,customer_service,add_services,reputation
ISP,,,,,,,,
Provider1,100,50,20,20,4.5,4.0,3.5,4.3
Provider2,150,75,15,25,4.7,4.2,4.0,4.5
Provider3,200,100,10,30,4.8,4.6,4.5,4.7
Provider4,50,25,30,15,4.2,3.8,3.0,3.9
Provider5,120,60,18,22,4.6,4.1,3.8,4.4
Provider6,80,40,25,18,4.3,3.9,3.2,4.0
Provider7,250,125,8,35,4.9,4.7,4.6,4.8
Provider8,90,45,22,19,4.4,4.0,3.4,4.2
Provider9,70,35,28,17,4.1,3.7,3.1,3.8


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, Provider1 to Provider10
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   d_speed           10 non-null     int64  
 1   u_speed           10 non-null     int64  
 2   latency           10 non-null     int64  
 3   cost              10 non-null     int64  
 4   reliability       10 non-null     float64
 5   customer_service  10 non-null     float64
 6   add_services      10 non-null     float64
 7   reputation        10 non-null     float64
dtypes: float64(4), int64(4)
memory usage: 720.0+ bytes


#### Нормалізація значень критеріїв в залежності від їх типу (максимізовані/мінімізовані)

In [4]:
# Типи критеріїв
criteria_types = {
    "d_speed": "Maximized",
    "u_speed": "Maximized",
    "latency": "Minimized",
    "cost": "Minimized",
    "reliability": "Maximized",
    "customer_service": "Maximized",
    "add_services": "Maximized",
    "reputation": "Maximized",
}

# Вагові коефіцієнти критеріїв
weights = {
    "d_speed": 0.20,
    "u_speed": 0.15,
    "latency": 0.15,
    "cost": 0.10,
    "reliability": 0.15,
    "customer_service": 0.10,
    "add_services": 0.05,
    "reputation": 0.10,
}

assert sum(weights.values()) == 1, "The sum of the weight coefficients must equal 1!"

def normalize_criteria(df: pd.DataFrame, weights: Dict[str, float], criteria_types: Dict[str, str]) -> pd.DataFrame:
    normalized_df = df.copy()

    for column in df.columns:
        if criteria_types[column] == "Maximized":
            normalized_df[column] = df[column] / df[column].max()
        elif criteria_types[column] == "Minimized":
            normalized_df[column] = df[column].min() / df[column]

    return normalized_df * pd.Series(weights)

normalized_df = normalize_criteria(df, weights, criteria_types)
normalized_df

,d_speed,u_speed,latency,cost,reliability,customer_service,add_services,reputation
ISP,,,,,,,,
Provider1,0.080,0.060,0.060000,0.075000,0.137755,0.085106,0.038043,0.089583
Provider2,0.120,0.090,0.080000,0.060000,0.143878,0.089362,0.043478,0.093750
Provider3,0.160,0.120,0.120000,0.050000,0.146939,0.097872,0.048913,0.097917
Provider4,0.040,0.030,0.040000,0.100000,0.128571,0.080851,0.032609,0.081250
Provider5,0.096,0.072,0.066667,0.068182,0.140816,0.087234,0.041304,0.091667
Provider6,0.064,0.048,0.048000,0.083333,0.131633,0.082979,0.034783,0.083333
Provider7,0.200,0.150,0.150000,0.042857,0.150000,0.100000,0.050000,0.100000
Provider8,0.072,0.054,0.054545,0.078947,0.134694,0.085106,0.036957,0.087500
Provider9,0.056,0.042,0.042857,0.088235,0.125510,0.078723,0.033696,0.079167


#### Обчислення інтегральних оцінок для кожного ISP

In [5]:
def calculate_integral_score(normalized_df: pd.DataFrame) -> pd.Series:
    return normalized_df.sum(axis=1)

scores = calculate_integral_score(normalized_df)
scores

ISP
Provider1     0.625488
Provider2     0.720468
Provider3     0.841641
Provider4     0.533281
Provider5     0.663870
Provider6     0.576061
Provider7     0.942857
Provider8     0.603750
Provider9     0.546188
Provider10    0.782424
dtype: float64

#### Знаходження ISP з найвищою інтегральною оцінкою (оптимального варіанту)

In [6]:
def find_optimal(scores: pd.Series) -> str:
    return scores.idxmax()

optimal_provider = find_optimal(scores)
print(f"Optimal provider: {optimal_provider}")

Optimal provider: Provider7


## Додаткове завдання

In [7]:
supplies = [40, 60]    # Запаси в пунктах A1 та A2
demands = [30, 50, 20] # Потреби в пунктах B1, B2 та B3
# Вартість транспортування з кожного пункту відправлення до кожного пункту призначення
costs = [
    [2, 3, 1],  # Витрати з пункту A1 -> B1, B2, B3
    [4, 2, 3],  # Витрати з пункту A2 -> B1, B2, B3
]

num_sources, num_destinations = len(supplies), len(demands)
solver = pywraplp.Solver.CreateSolver("GLOP")

x = {}
for i in range(num_sources):
    for j in range(num_destinations):
        x[i, j] = solver.NumVar(0, solver.infinity(), f"x_{i}_{j}")

# Обмеження по постачаннях
for i in range(num_sources):
    solver.Add(solver.Sum([x[i, j] for j in range(num_destinations)]) <= supplies[i])

# Обмеження по потребах
for j in range(num_destinations):
    solver.Add(solver.Sum([x[i, j] for i in range(num_sources)]) >= demands[j])

# Мети: мінімізувати загальні витрати
objective = solver.Sum([costs[i][j] * x[i, j] for i in range(num_sources) for j in range(num_destinations)])
solver.Minimize(objective)

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print(f"Objective value (Total expenses) = {solver.Objective().Value():0.1f}")

    for i in range(num_sources):
        for j in range(num_destinations):
            quantity = int(x[i, j].solution_value())
            if quantity > 0:
                print(f"A{i + 1} -> B{j + 1} {quantity} units × {costs[i][j]}")
else:
    print("The problem does not have an optimal solution")

Objective value (Total expenses) = 200.0
A1 -> B1 20 units × 2
A1 -> B3 20 units × 1
A2 -> B1 10 units × 4
A2 -> B2 50 units × 2
